In [9]:
class ConjunctiveNormalForm:
        
    def __init__(self, *clauses):
        self.numero_variables = 0
        self.longitud = 0
        self.clausulas = []
        self.variables = set([])
        if clauses:
            self.insert(*clauses)
        
    def variable_count(self):
        return len(self.variables)

    def __len__(self):
        return len(self.clausulas) #self.longitud

    def insert(self, *clauses):
        for clausula in clauses:
            self.clausulas.append(clausula)
            self.longitud += 1
            
            for positivas in clausula[0]: 
                self.variables.add(positivas)
                
            for negativas in clausula[1]:    
                self.variables.add(negativas)
                
            
    def __getitem__(self, i):
        return self.clausulas[i]
        
    def evalue(self, truth):        
        #Para que la expresion sea cierta, todas las clausualas deben ser ciertas
        for clausula in self.clausulas:
            #Si el conjunto de las ciertas y las clausulas positivas es cero
            #es porque no tienen ninguna en comun: todas las variables positivas
            #estan negadas
            if len(set(clausula[0]).intersection(truth)) == 0:
                #En dicho caso, comprobamos con las variables negativas. 
                #Si la longitud de la union es igual a longitud de las clausulas negativas
                #es porque todas aparecen como positivas, haciendo falsa la expresion
                #Si la longitud es distinta, alguna variable negada esta como falsa, 
                #haciendo verdadera la clausula
                if len(set(clausula[1]).intersection(truth)) == len(clausula[1]):
                    return False
                        
        return True

    def satisfiable(self):               
        resultado = []
        

In [15]:
import networkx as nx

def reduction_to_clique(fnc):
    resultado = []    
    for i in range(len(fnc)):
        clausula_actual = fnc[i]
        for j in range(i+1, len(fnc)):
            if i!=j:
                clausula_destino = fnc[j]
                for positiva_origen in clausula_actual[0]:
                    for positiva_destino in clausula_destino[0]:
                        resultado.append([(str(i)+':'+positiva_origen), (str(j)+':'+positiva_destino)])
                    for negativa_destino in clausula_destino[1]:
                        if positiva_origen != negativa_destino:
                            resultado.append([(str(i)+':'+positiva_origen),(str(j)+':!'+negativa_destino)])
    
                for negativa_origen in clausula_actual[1]:
                    for negativa_destino in clausula_destino[1]:
                        resultado.append([(str(i)+':!'+negativa_origen),(str(j)+':!'+negativa_destino)])
                    for positiva_destino in clausula_destino[0]:
                        if negativa_origen != positiva_destino:
                            resultado.append([(str(i)+':!'+negativa_origen), (str(j)+':'+positiva_destino)])
    
    return nx.Graph(resultado)

In [18]:
def graphs_are_equals(g1, g2):
   
    if set(g1.nodes()) != set(g2.nodes()):
        return False
      
    for a, b in [(g1, g2), (g2, g1)]:
        for u, v in a.edges_iter():
            if not b.has_edge(u, v):
                return False
            
    return True